In [8]:
import psycopg2
import pandas as pd

In [9]:
''' Аттрибуты подключения содержатся в отдельном файле для безопасности.
    Импортируются в словарь, который далее используется при подключении
    и запросах к базе.
'''
import creds as c_

CONN_ATTR = {
    'user': c_.USER,
    'password': c_.PASSWORD,
    'host': c_.HOST,
    'port': c_.PORT,
    'dbname': c_.DBNAME
    }

def make_request(query_: str) -> object:
    ''' Делает запросы к БД с определёнными аттрибутами
    Arguments:
        query_ [str] -- Строка SQL-запроса (можно многострочник)
    Returns:
        [object] -- Ответ на запрос
    '''
    with psycopg2.connect(**CONN_ATTR) as conn:
        return pd.read_sql_query(query_, conn)

### Задание 3.1 ###

Сколько вакансий есть в базе?

***Ответ - 49197***

In [10]:
query_3_1 = f'''
SELECT COUNT(*)
FROM public.vacancies
'''
display(make_request(query_3_1))

c:\programs\python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,49197


### Задание 3.2 ###

Сколько работодателей?

***Ответ - 23501***

In [11]:
query_3_2 = f'''
SELECT COUNT(*)
FROM public.employers
'''
display(make_request(query_3_2))

c:\programs\python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,23501


### Задание 3.3 ###

Сколько регионов?

***Ответ - 1362***

In [12]:
query_3_2 = f'''
SELECT COUNT(*)
FROM public.areas
'''
display(make_request(query_3_2))

c:\programs\python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,1362


### Задание 3.3 ###

Сколько сфер деятельности?

***Ответ - 294***

In [13]:
query_3_4 = f'''
SELECT COUNT(*)
FROM public.industries
'''
display(make_request(query_3_4))

c:\programs\python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,294
